### Using a GitHub Personal Access Token (PAT) to Push from a SageMaker Notebook

When working in SageMaker notebooks, you may often need to push code updates to GitHub repositories. However, SageMaker notebooks are typically launched with temporary instances that don’t persist configurations, including SSH keys, across sessions. This makes HTTPS-based authentication, secured with a GitHub Personal Access Token (PAT), a practical solution. PATs provide flexibility for authentication and enable seamless interaction with both public and private repositories directly from your notebook. 

> **Important Note**: Personal access tokens are powerful credentials that grant specific permissions to your GitHub account. To ensure security, only select the minimum necessary permissions and handle the token carefully.


### Step 1: Generate a Personal Access Token (PAT) on GitHub

1. Go to **Settings > Developer settings > Personal access tokens** on GitHub.
2. Click **Generate new token**, select **Classic**.
3. Give your token a descriptive name (e.g., "SageMaker Access Token") and set an expiration date if desired for added security.
4. **Select the minimum permissions needed**:
   - **For public repositories**: Choose only **`public_repo`**.
   - **For private repositories**: Choose **`repo`** (full control of private repositories).
   - Optional permissions, if needed:
     - **`repo:status`**: Access commit status (if checking status checks).
     - **`workflow`**: Update GitHub Actions workflows (only if working with GitHub Actions).
5. Generate the token and **copy it** (you won’t be able to see it again).

> **Caution**: Treat your PAT like a password. Avoid sharing it or exposing it in your code. Store it securely (e.g., via a password manager like LastPass) and consider rotating it regularly.


### Step 2: Configure Git `user.name` and `user.email`
In your SageMaker or Jupyter notebook environment, run the following commands to set up your Git user information


In [44]:

!git config --global user.name "Chris Endemann"
!git config --global user.email endeman@wisc.edu


### Explanation

- **`user.name`**: This is your GitHub username, which will appear in the commit history as the author of the changes.
- **`user.email`**: This should match the email associated with your GitHub account so that commits are properly linked to your profile.

Setting this globally (`--global`) will ensure the configuration persists across all repositories in the environment. If you’re working in a temporary environment, you may need to re-run this configuration after a restart.

### Step 3: Use `getpass` to Prompt for Username and PAT

The `getpass` library allows you to input your GitHub username and PAT without exposing them in the notebook. This approach ensures you’re not hardcoding sensitive information.


In [45]:
import getpass

# Prompt for GitHub username and PAT securely
# github_url = 'github.com/UW-Madison-DataScience/test_AWS.git' # found under Code -> Clone -> HTTPS (remote the https:// before the rest of the address)
# username = input("GitHub Username: ")
# token = getpass.getpass("GitHub Personal Access Token (PAT): ")

**Note**: After running, you may want to comment out the above code so that you don't have to enter in your login every time you run your whole notebook


### Explanation

- **`input("GitHub Username: ")`**: Prompts you to enter your GitHub username.
- **`getpass.getpass("GitHub Personal Access Token (PAT): ")`**: Prompts you to securely enter the PAT, keeping it hidden on the screen.



### Step 4: Add, Commit, and Push Changes with Manual Authentication
1. **Navigate to the Repository Directory** (adjust the path if needed):


In [32]:
!pwd
# !cd test_AWS

/home/ec2-user/SageMaker/test_AWS



2. **Add and Commit Changes**:



In [53]:
!git add .
!git commit -m "Added updates from Jupyter notebook"


[main 637d64c] Added updates from Jupyter notebook
 1 file changed, 19 insertions(+), 27 deletions(-)


3. **Pull the Latest Changes from the Main Branch**: Pull the latest changes from the remote main branch to ensure your local branch is up-to-date.

    Recommended: Set the Pull Strategy for this Repository (Merge by Default)

    All options:

    * Merge (pull.rebase false): Combines the remote changes into your local branch as a merge commit.
    * Rebase (pull.rebase true): Replays your local changes on top of the updated main branch, resulting in a linear history.
    * Fast-forward only (pull.ff only): Only pulls if the local branch can fast-forward to the remote without diverging (no new commits locally).

In [51]:
!git config pull.rebase false # Combines the remote changes into your local branch as a merge commit.

!git pull origin main


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 2), reused 3 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 1.18 KiB | 1.18 MiB/s, done.
From https://github.com/UW-Madison-DataScience/test_AWS
 * branch            main       -> FETCH_HEAD
   ebe991c..adfe7b1  main       -> origin/main
Updating 2ff6cf0..adfe7b1
Fast-forward
 .gitignore                                         |    3 +
 ...storage-and-access-via-buckets-checkpoint.ipynb |    6 -
 .../01_Intro-train-models-checkpoint.ipynb         | 2555 --------------------
 .../02_Hyperparameter-tuning-checkpoint.ipynb      |  470 ----
 .../create_large_data-checkpoint.ipynb             |   54 -
 .../push-git-updates-checkpoint.ipynb              |  322 ---
 .ipynb_checkpoints/train_nn-checkpoint.py          |   93 -
 .ipynb_checkpoints/train_xgboost-checkpoint.py     |   65 -
 __pycache__/train_xgboost.cpython-310.py

If you get merge conflicts, be sure to resolve those before moving forward (e.g., use git checkout -> add -> commit). You can skip the below code if you don't have any conflicts. 

In [52]:
# Keep your local changes in one conflicting file
# !git checkout --ours train_nn.py

# Keep remote version for the other conflicting file
# !git checkout --theirs train_xgboost.py

# # Stage the files to mark the conflicts as resolved
# !git add train_nn.py
# !git add train_xgboost.py

# # Commit the merge result
# !git commit -m "Resolved merge conflicts by keeping local changes"

4. **Push Changes and Enter Credentials**:

In [54]:
# Push with embedded credentials from getpass (avoids interactive prompt)
!git push https://{username}:{token}@{github_url} main

 

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 780 bytes | 780.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/UW-Madison-DataScience/test_AWS.git
   adfe7b1..637d64c  main -> main
